In [152]:
import pandas as pd
import requests
import json
import gmaps
from pprint import pprint

# Google developer API key
# from config import gkey


In [153]:
redfin_df = pd.read_csv('redfin_2020-05-30-16-15-45.csv')
redfin_df
redfin_closer_df = pd.read_csv('redfin_2020-05-31-12-00-06.csv')
redfin_closer_df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Single Family Residential,246 Monte Vista Ln,DALY CITY,CA,94015,898000,3,2.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Daly-City/246-Monte-V...,MLSListings,ML81794298,Y,Y,37.658315,-122.463216
1,MLS Listing,NaN,Single Family Residential,1950 Evergreen Ave,San Leandro,CA,94577,799888,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/San-Leandro/1950-Ever...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40905479,N,Y,37.722627,-122.136859
2,MLS Listing,NaN,Single Family Residential,9648 Dunbar Dr,Oakland,CA,94603,669880,3,2.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40904884,N,Y,37.741184,-122.178730
3,MLS Listing,NaN,Single Family Residential,2543 Lancaster Rd,Hayward,CA,94542,874950,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/2543-Lancaste...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906691,N,Y,37.664743,-122.051583
4,MLS Listing,NaN,Single Family Residential,1021 E 23rd St,Oakland,CA,94606,699000,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40906683,N,Y,37.797502,-122.241082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,New Construction Plan,NaN,Single Family Residential,Stellar 820,San Francisco,CA,94123,254390,2,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,NewHomeSource.com,Plan-121706983,N,Y,37.798864,-122.437717
173,New Construction Plan,NaN,Single Family Residential,Residence 3 WLH,Richmond,CA,94804,595000,3,3.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,NewHomeSource.com,Plan-121781294,N,Y,37.920715,-122.355477
174,New Construction Plan,NaN,Single Family Residential,Plan 5 WLH,Hayward,CA,94544,800000,4,3.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,NewHomeSource.com,Plan-121781292,N,Y,37.632475,-122.049066
175,New Construction Plan,NaN,Single Family Residential,Plan 3 Modeled,Hayward,CA,94544,769000,4,3.5,...,Active,NaN,NaN,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,NewHomeSource.com,Plan-111707850,N,Y,37.647801,-122.064880


In [94]:
#rename URL column for obvious reasons
redfin_df2 = redfin_df.rename(columns={"URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)":"URL"})
redfin_closer_df2 = redfin_closer_df.rename(columns={"URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)":"URL"})


In [101]:
#Pulling out only necessary columns 
locations = pd.DataFrame(redfin_df2[["ADDRESS", "CITY", "URL", "LATITUDE", "LONGITUDE"]])
locations

locations2 = pd.DataFrame(redfin_closer_df2[["ADDRESS", "CITY", "URL", "LATITUDE", "LONGITUDE"]])
locations2


,ADDRESS,CITY,URL,LATITUDE,LONGITUDE
0,246 Monte Vista Ln,DALY CITY,http://www.redfin.com/CA/Daly-City/246-Monte-V...,37.658315,-122.463216
1,1950 Evergreen Ave,San Leandro,http://www.redfin.com/CA/San-Leandro/1950-Ever...,37.722627,-122.136859
2,9648 Dunbar Dr,Oakland,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,37.741184,-122.178730
3,2543 Lancaster Rd,Hayward,http://www.redfin.com/CA/Hayward/2543-Lancaste...,37.664743,-122.051583
4,1021 E 23rd St,Oakland,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,37.797502,-122.241082
...,...,...,...,...,...
172,Stellar 820,San Francisco,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,37.798864,-122.437717
173,Residence 3 WLH,Richmond,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,37.920715,-122.355477
174,Plan 5 WLH,Hayward,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,37.632475,-122.049066
175,Plan 3 Modeled,Hayward,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,37.647801,-122.064880


In [143]:
#Add column with lat and long combined 
locations["LOCATION"] = locations["LATITUDE"].astype(str) +"," + locations["LONGITUDE"].astype(str)

locations2["LOCATION"] = locations2["LATITUDE"].astype(str) +"," + locations2["LONGITUDE"].astype(str)

locations2["travel_time"] = ""
locations2["travel_cost"] = ""


In [103]:
locations.head(30)

,ADDRESS,CITY,URL,LATITUDE,LONGITUDE,LOCATION
0,1994 Corsica Way,Brentwood,http://www.redfin.com/CA/Brentwood/1994-Corsic...,37.897804,-121.720147,"37.897804,-121.72014709999999"
1,2613 Forty Niner Way,Antioch,http://www.redfin.com/CA/Antioch/2613-Forty-Ni...,37.958918,-121.778712,"37.95891760000001,-121.7787118"
2,332 Blue Lake Cir,Oakley,http://www.redfin.com/CA/Oakley/332-Blue-Lake-...,37.988247,-121.633457,"37.9882465,-121.6334572"
3,516 Patterson Blvd,Pleasant Hill,http://www.redfin.com/CA/Pleasant-Hill/516-Pat...,37.936191,-122.071916,"37.9361913,-122.0719155"
4,4911 Ridgefield Cir,Fairfield,http://www.redfin.com/CA/Fairfield/4911-Ridgef...,38.166743,-122.139906,"38.1667429,-122.13990600000001"
5,246 Monte Vista Ln,DALY CITY,http://www.redfin.com/CA/Daly-City/246-Monte-V...,37.658315,-122.463216,"37.658315,-122.46321599999999"
6,1401 Cypress Dr,Tracy,http://www.redfin.com/CA/Tracy/1401-Cypress-Dr...,37.733948,-121.441400,"37.7339485,-121.44139950000002"
7,1950 Evergreen Ave,San Leandro,http://www.redfin.com/CA/San-Leandro/1950-Ever...,37.722627,-122.136859,"37.722626500000004,-122.13685929999998"
8,84 Heritage Way,Brentwood,http://www.redfin.com/CA/Brentwood/84-Heritage...,37.924334,-121.679238,"37.9243343,-121.6792383"
9,9648 Dunbar Dr,Oakland,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,37.741184,-122.178730,"37.74118370000001,-122.1787301"


In [144]:
locations2.head(30)

,ADDRESS,CITY,URL,LATITUDE,LONGITUDE,LOCATION,travel_time,travel_cost
0,246 Monte Vista Ln,DALY CITY,http://www.redfin.com/CA/Daly-City/246-Monte-V...,37.658315,-122.463216,"37.658315,-122.46321599999999",,
1,1950 Evergreen Ave,San Leandro,http://www.redfin.com/CA/San-Leandro/1950-Ever...,37.722627,-122.136859,"37.722626500000004,-122.13685929999998",,
2,9648 Dunbar Dr,Oakland,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,37.741184,-122.178730,"37.74118370000001,-122.1787301",,
3,2543 Lancaster Rd,Hayward,http://www.redfin.com/CA/Hayward/2543-Lancaste...,37.664743,-122.051583,"37.6647434,-122.0515834",,
4,1021 E 23rd St,Oakland,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,37.797502,-122.241082,"37.7975023,-122.2410824",,
5,1867 Pleasant Hill Rd,Pleasant Hill,http://www.redfin.com/CA/Pleasant-Hill/1867-Pl...,37.933864,-122.085453,"37.9338641,-122.08545330000001",,
6,1060 69Th Ave,Oakland,http://www.redfin.com/CA/Oakland/1060-69th-Ave...,37.758351,-122.193427,"37.758351,-122.193427",,
7,10 Harbor View Dr,Richmond,http://www.redfin.com/CA/Richmond/10-Harbor-Vi...,37.908416,-122.349211,"37.9084159,-122.3492108",,
8,14869 Midland Rd,San Leandro,http://www.redfin.com/CA/San-Leandro/14869-Mid...,37.712603,-122.123350,"37.712603200000004,-122.1233501",,
9,26732 Gaither Way,Hayward,http://www.redfin.com/CA/Hayward/26732-Gaither...,37.638613,-122.082369,"37.6386135,-122.0823686",,


In [105]:
#google dependencies
# target_coordinates = locations["LOCATION"]


In [145]:
#params



outputFormat = "json"
# outoutFormat = "xml"

base_url = f"https://maps.googleapis.com/maps/api/directions/{outputFormat}?{params}&key={gkey}"
    

    


In [149]:
params = {
#     "destination": "37.29014229999999,-121.8443185",
    "origin": "37.776501,-122.417018",
    "mode": "transit"
}

# use iterrows to iterate through pandas dataframe
for index, row in locations2.iterrows():

    # get restaurant type from df
    house_loc = row['LOCATION']

    # add keyword to params dict
    params['destination'] = house_loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {house_loc}.")
    response = requests.get(base_url, params=params).json()


    # pprint(response)

    # extract results
    try:
        results = response['routes'][0]['fare']['text']

        results2 = response['routes'][0]['legs'][0]['duration']['text']

        locations2.loc[index, 'travel_cost'] = results
        locations2.loc[index, 'travel_time'] = results2

        print(f"Travel time from location is: {results2}")
        print(f"Cost for travel is: {results}")
        print("------------")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print("------------")

Retrieving Results for Index 0: 37.658315,-122.46321599999999.
Travel time from location is: 1 hour 9 mins
Cost for travel is: $6.65
------------
Retrieving Results for Index 1: 37.722626500000004,-122.13685929999998.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: 37.74118370000001,-122.1787301.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: 37.6647434,-122.0515834.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: 37.7975023,-122.2410824.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: 37.9338641,-122.08545330000001.
Travel time from location is: 9 hours 41 mins
Cost for travel is: $9.30
------------
Retrieving Results for Index 6: 37.758351,-122.193427.
Travel time from location is: 40 mins
Cost for travel is: $5.00
------------
Retrieving Results for Index 7: 37.9084159,-122.3492108.
Missing field/result... skipping.
------------
Retrieving Results for Index 8:

Missing field/result... skipping.
------------
Retrieving Results for Index 72: 37.707702000000005,-122.13444150000001.
Missing field/result... skipping.
------------
Retrieving Results for Index 73: 37.6918824,-122.0787956.
Travel time from location is: 45 mins
Cost for travel is: $5.95
------------
Retrieving Results for Index 74: 37.7665854,-122.19001399999999.
Missing field/result... skipping.
------------
Retrieving Results for Index 75: 37.7949404,-122.2372709.
Missing field/result... skipping.
------------
Retrieving Results for Index 76: 37.7053597,-122.1185582.
Travel time from location is: 58 mins
Cost for travel is: $5.60
------------
Retrieving Results for Index 77: 37.915031400000004,-122.3180814.
Travel time from location is: 59 mins
Cost for travel is: $5.30
------------
Retrieving Results for Index 78: 37.6787435,-122.05335929999998.
Missing field/result... skipping.
------------
Retrieving Results for Index 79: 37.6952872,-122.1197562.
Travel time from location is: 47 

Travel time from location is: 1 hour 0 mins
Cost for travel is: $5.95
------------
Retrieving Results for Index 141: 37.723611600000005,-122.1330662.
Missing field/result... skipping.
------------
Retrieving Results for Index 142: 37.7673745,-122.19867079999999.
Missing field/result... skipping.
------------
Retrieving Results for Index 143: 37.8004466,-122.23217379999998.
Missing field/result... skipping.
------------
Retrieving Results for Index 144: 37.7930161,-122.21439879999998.
Missing field/result... skipping.
------------
Retrieving Results for Index 145: 37.7340599,-122.17232109999999.
Missing field/result... skipping.
------------
Retrieving Results for Index 146: 37.796034399999996,-122.24634520000001.
Missing field/result... skipping.
------------
Retrieving Results for Index 147: 37.71602120000001,-122.1510321.
Travel time from location is: 50 mins
Cost for travel is: $5.40
------------
Retrieving Results for Index 148: 37.794427500000005,-122.1970643.
Missing field/result

In [150]:
locations2

,ADDRESS,CITY,URL,LATITUDE,LONGITUDE,LOCATION,travel_time,travel_cost
0,246 Monte Vista Ln,DALY CITY,http://www.redfin.com/CA/Daly-City/246-Monte-V...,37.658315,-122.463216,"37.658315,-122.46321599999999",1 hour 9 mins,$6.65
1,1950 Evergreen Ave,San Leandro,http://www.redfin.com/CA/San-Leandro/1950-Ever...,37.722627,-122.136859,"37.722626500000004,-122.13685929999998",,
2,9648 Dunbar Dr,Oakland,http://www.redfin.com/CA/Oakland/9648-Dunbar-D...,37.741184,-122.178730,"37.74118370000001,-122.1787301",,
3,2543 Lancaster Rd,Hayward,http://www.redfin.com/CA/Hayward/2543-Lancaste...,37.664743,-122.051583,"37.6647434,-122.0515834",,
4,1021 E 23rd St,Oakland,http://www.redfin.com/CA/Oakland/1021-E-23rd-S...,37.797502,-122.241082,"37.7975023,-122.2410824",,
...,...,...,...,...,...,...,...,...
172,Stellar 820,San Francisco,http://www.redfin.com/CA/San-Francisco/Gaia-Ho...,37.798864,-122.437717,"37.798864,-122.43771699999999",18 mins,$4.50
173,Residence 3 WLH,Richmond,http://www.redfin.com/CA/Richmond/Rows-at-NOMA...,37.920715,-122.355477,"37.920715,-122.35547700000001",,
174,Plan 5 WLH,Hayward,http://www.redfin.com/CA/Hayward/SoHay-Line/Pl...,37.632475,-122.049066,"37.632475,-122.049066",1 hour 4 mins,$6.25
175,Plan 3 Modeled,Hayward,http://www.redfin.com/CA/Hayward/Motion-at-Mis...,37.647801,-122.064880,"37.6478013,-122.06487959999998",,


In [151]:
locations2.to_csv("travel_time_redfin.csv")